In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_json("vietnam_travel_dataset.json")

In [3]:
df.head()

,id,type,name,region,description,best_time_to_visit,tags,semantic_text,connections,city
0,city_hanoi,City,Hanoi,Northern Vietnam,Hanoi is located in Northern Vietnam. It’s kno...,February to May,"[culture, food, heritage]","Hanoi offers a mix of culture, food, heritage ...","[{'relation': 'Connected_To', 'target': 'city_...",NaN
1,city_ha_long,City,Ha Long Bay,Northern Vietnam,Ha Long Bay is located in Northern Vietnam. It...,February to May,"[beach, cruise, nature]","Ha Long Bay offers a mix of beach, cruise, nat...","[{'relation': 'Connected_To', 'target': 'city_...",NaN
2,city_sapa,City,Sapa,Northern Vietnam,Sapa is located in Northern Vietnam. It’s know...,October to April,"[mountain, trekking, ethnic]","Sapa offers a mix of mountain, trekking, ethni...","[{'relation': 'Connected_To', 'target': 'city_...",NaN
3,city_hue,City,Hue,Central Vietnam,Hue is located in Central Vietnam. It’s known ...,October to April,"[imperial, history, architecture]","Hue offers a mix of imperial, history, archite...","[{'relation': 'Connected_To', 'target': 'city_...",NaN
4,city_hoi_an,City,Hoi An,Central Vietnam,Hoi An is located in Central Vietnam. It’s kno...,October to April,"[lanterns, romantic, heritage]","Hoi An offers a mix of lanterns, romantic, her...","[{'relation': 'Connected_To', 'target': 'city_...",NaN


In [6]:
df.shape

(360, 10)

In [4]:
df['type'].unique()

array(['City', 'Attraction', 'Hotel', 'Activity'], dtype=object)

In [5]:
df['region'].unique()

array(['Northern Vietnam', 'Central Vietnam', 'Southern Vietnam', nan],
      dtype=object)

In [10]:
df['connections'].tail(1)

359    [{'relation': 'Available_In', 'target': 'city_...
Name: connections, dtype: object

In [3]:
# visualize_graph.py
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx
import config

NEO_BATCH = 500  # number of relationships to fetch / visualize

driver = GraphDatabase.driver(config.NEO4J_URI, auth=(config.NEO4J_USERNAME, config.NEO4J_PASSWORD))

def fetch_subgraph(tx, limit=500):
    # fetch nodes and relationships up to a limit
    q = (
        "MATCH (a:Entity)-[r]->(b:Entity) "
        "RETURN a.id AS a_id, labels(a) AS a_labels, a.name AS a_name, "
        "b.id AS b_id, labels(b) AS b_labels, b.name AS b_name, type(r) AS rel "
        "LIMIT $limit"
    )
    return list(tx.run(q, limit=limit))

def build_pyvis(rows, output_html="neo4j_viz.html"):
    net = Network(height="900px", width="100%", notebook=False, directed=True)
    for rec in rows:
        a_id = rec["a_id"]; a_name = rec["a_name"] or a_id
        b_id = rec["b_id"]; b_name = rec["b_name"] or b_id
        a_labels = rec["a_labels"]; b_labels = rec["b_labels"]
        rel = rec["rel"]

        net.add_node(a_id, label=f"{a_name}\n({','.join(a_labels)})", title=f"{a_name}")
        net.add_node(b_id, label=f"{b_name}\n({','.join(b_labels)})", title=f"{b_name}")
        net.add_edge(a_id, b_id, title=rel)

    net.show(output_html, notebook=False)
    print(f"Saved visualization to {output_html}")

def main():
    with driver.session() as session:
        rows = session.execute_read(fetch_subgraph, limit=NEO_BATCH)
    build_pyvis(rows)

if __name__ == "__main__":
    main()


neo4j_viz.html
Saved visualization to neo4j_viz.html


In [10]:
pip install sentence_transformers

  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.9.0-cp311-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached scikit_learn-1.7.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.2-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.35.3-py3-none-any.whl (564 kB)
Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
Using cached torch-2.9.0-cp311-none-macosx_11_0_arm64.whl (74.5 MB)
Using cached scikit_learn-1.7.2-cp311-cp311-macosx_12_0_arm64.whl (8.6 MB)
Using cached scipy-1.16.2-cp311-cp311-macosx_14_0_arm6